In [1]:
help(list.count)

Help on method_descriptor:

count(self, value, /)
    Return number of occurrences of value.



In [2]:
help(str.count)

Help on method_descriptor:

count(...)
    S.count(sub[, start[, end]]) -> int
    
    Return the number of non-overlapping occurrences of substring sub in
    string S[start:end].  Optional arguments start and end are
    interpreted as in slice notation.



In [3]:
help(tuple.count)

Help on method_descriptor:

count(self, value, /)
    Return number of occurrences of value.



In [5]:
import pandas as pd

In [9]:
army_list1 = ['Alpha', 'Beta', 'S', 'Mike', 'Delta', 'Zulu', 'x-ray', 'yankee']
army_list2 = ['Alpha', 'Charlie', 'A', 'November','Delta']

i=0
while(i<max(len(army_list1[:]),len(army_list2[:]))):
    
    if(army_list1[i]==army_list2[i]):
        print('removing ', army_list1[i])
        army_list1.pop(i)
        army_list2.pop(i)
        
    elif(army_list1[i]=="S" and army_list2[i]=="A"):
        print('removing ', army_list1[i], 'from left')
        army_list1.pop(i)
        
    elif(army_list1[i]=="A" and army_list2[i]=="S"):
        print('removing ', army_list2[i], 'from left')
        army_list2.pop(i)
 
    
    i+=1

print(army_list1)
print(army_list2)

removing  Alpha
removing  S from left


IndexError: list index out of range

In [2]:
army_list1

['Beta', 'Mike', 'Delta']

In [3]:
army_list2

['Charlie', 'A', 'November', 'Delta']

In [10]:
army_list1 = ['Alpha', 'Beta', 'S', 'Mike', 'Delta']
army_list2 = ['Alpha', 'Charlie', 'A', 'November','Delta']

new_list1 = []
new_list2 = []

for left, right in zip(army_list1, army_list2):
    if left == "S" and  right == "A":
        new_list1.append(right)     
    elif left == "A" and right == "S":
        new_list2.append(left)
    elif not left == right:
        new_list1.append(left)
        new_list2.append(right)

print(new_list1)
print(new_list2)
        

['Beta', 'A', 'Mike']
['Charlie', 'November']


In [12]:
import bisect

age_borders = 2, 4, 13, 20, 65
age_descriptions = ('Baby', 'Toddler', 'Kid', 
                    'Teenager', 'Adult', 'Old Geezer')

age = float(input('And the age is ? '))

age_index = bisect.bisect_right(age_borders, age)
print(f'This is a {age_descriptions[age_index]}')

And the age is ? 65
This is a Old Geezer


In [13]:
def hello(func):
    def inner():
        print('Hello')
        return func()
    return inner

def goodbye(func):
    def inner():
        result = func()
        print('Goodbye')
        return result
    return inner

@hello
@goodbye
def a():
    print('I am "a"')

@hello
@goodbye
def b():
    print('I am "b"')

In [14]:
a()
b()

Hello
I am "a"
Goodbye
Hello
I am "b"
Goodbye


In [2]:
AFFIRMATION = frozenset({'y', 'yes', 'yeh', 'k', 'si', 'ok', 'yeah', 'yup'})
REJECTION = frozenset({'n', 'no', 'nope', 'nah', 'exit', 'quit', 'bye'})

def yesno(msg: str = "Yes or No? ") -> bool :
    while True:  # valid response loop
        response = input(msg)
        if response:
            if response.lower() in AFFIRMATION:
                return True
            elif response.lower() in REJECTION:
                return False
        print('Please answer Yes or No.')

In [4]:
print(yesno('Testing: '))

Testing: nope
False


In [3]:
word = input()
best = ''
alphabet = 'abcdefghijklmnopqrstuvwxyz'

for n in range(2):
    sub = word[:-2+n]
    for _ in alphabet:
        if _ in sub:
            ind = 0
            while ind < len(sub):
                if sub[ind] == _:
                    best += sub[:ind+1][::-1]
                    word = word[ind+1:]
                    break
                ind += 1
            break

best += word[::-1]
print(best)

anakonda
aanadnok


In [11]:
    bored = False
    while not bored:
        age_str = input('Age? (enter alone to exit) ')
        if not age_str:
            bored = True
            continue
        try:
            age = int(age_str)
        except ValueError as e:
            print('Please enter a whole number')
        else:
            result = age >= 18
            print('Adult?', result)

Age? (enter alone to exit) ggg
Please enter a whole number
Age? (enter alone to exit) 45.6
Please enter a whole number
Age? (enter alone to exit) 17
Adult? False
Age? (enter alone to exit) 20
Adult? True
Age? (enter alone to exit) 


In [24]:
'''Hello! I'm very new to Python, I actually just started a few days ago and I'm mostly self teaching. 
I have to make an age calculator for my class and I've seem to have figured my own way to do it, 
however my professor wants us to use datetime.datime.now(), datetime.datetime(year), 
and datetime.datetime.timestamp() and begin the code using user_age_years = int(input'enter your age in years:\n')) user_age_days = user_age_years * 365 print('You are at least %d days old.' % user_age_days)
I dont necessarily know how to incorporate those into my code. 
If anyone can help simplify my code and use those commands that would be great. 
Also, ignore the comments with URLs that's my way of citing my source 
(yes I know it's not the proper comment symbol)
'''

from datetime import date

today = date.today()

def get_int(start, fini, msg):
    ''' prompt user with msg for an integer within define range '''
    while True:
        response = input(msg)
        if response:
            try:
                num = int(response)
            except ValueError as e:
                pass
            else:
                if start <= num <= fini:
                    return num
        print(f'You must enter a whole number between {start} and {fini}')

def get_dob():
    ''' get date of birth from user and return age'''
    while True:
        year = get_int(today.year - 105, today.year, 'Year of birth: ')
        month = get_int(1, 12, 'Month of birth: ')
        day = get_int(1, 31, f'What day in month {month} were you born: ')
        try:
            dob = date(year, month, day)
        except:
            pass
        else:
            if dob.year < today.year:
                return dob
        print('I think you need to try that again.')
    
def calc_age(dob):
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))   

print("-----Welcome to the Age Calculator-----")
age = calc_age(get_dob())
days = age * 365
hours = days * 24
minutes = hours * 60
seconds = minutes * 60
print(f"You are at least {minutes} minutes {seconds} seconds old")

print("-----Welcome to the Heartbeat Calculator-----")
heartrate = 72 * 60 * 24 * 365
heartbeat = heartrate * age
print(f"Your heart has beaten around {heartbeat} times within your lifetime.")

print("-----Welcome to the Sneeze Calculator-----")
sneeze = 4 * 24 * 365
sneezeavg = sneeze * age
print(f"You have sneezed around {sneezeavg} times within your lifetime.")

print("-----Welcome to the Calorie Calculator-----")
cal = 2200 * 24 * 365
calavg = cal * age
print(f"You have burned around {calavg} calories within your lifetime.")

#comment: estimates the number of calories expended (avg: 2200)
bananas = float(105)
avgbananas = float(calavg) / bananas
avgbananas = round(avgbananas, 2)
print(f"That's the equivalent of about {avgbananas} bananas!")

print("-----Welcome to the Blinking Calculator-----")
blink = 1200 * 24 * 365
blinkavg = blink * age
print(f"You have blinked around {blinkavg} times within your lifetime.")

-----Welcome to the Age Calculator-----
Year of birth: 1963
Month of birth: 05
What day in month 5 were you born: 23
You are at least 28908000 minutes 1734480000 seconds old
-----Welcome to the Heartbeat Calculator-----
Your heart has beaten around 2081376000 times within your lifetime.
-----Welcome to the Sneeze Calculator-----
You have sneezed around 1927200 times within your lifetime.
-----Welcome to the Calorie Calculator-----
You have burned around 1059960000 calories within your lifetime.
That's the equivalent of about 10094857.14 bananas!
-----Welcome to the Blinking Calculator-----
You have blinked around 578160000 times within your lifetime.


In [27]:
num1=5
num2=3.2
print(f'{num1} x {num2} = {num1 * num2}')
print('{} x {} = {}'.format(num1, num2, num1 * num2))
print(f'{num1:4.1f} x {num2:4.1f} = {num1 * num2:6.1f}')
print('{:4.1f} x {:4.1f} = {:6.1f}'.format(num1, num2, num1 * num2))

5 x 3.2 = 16.0
5 x 3.2 = 16.0
 5.0 x  3.2 =   16.0
 5.0 x  3.2 =   16.0


In [28]:
items = []
while True:
    item = input('Next item: (enter to exit) ')
    if not item:
        break
    items.append(item)

print(items)

Next item: (enter to exit) f
Next item: (enter to exit) ghj
Next item: (enter to exit) o
Next item: (enter to exit) 
['f', 'ghj', 'o']


In [48]:
with open('/Users/stuartmoore/nums.txt') as nums:
    total = 0  # running total of valid inputs
    count = 0  # number of valid inputs
    for line_no, line in enumerate(nums, start=1):
        try:
            total += float(line.strip())
            count += 1
        except ValueError as e:
            print(f'line {line_no} has bad data: {line}')
if count > 0:
    print(f'Average is {total/count:.2f}')

line 1 has bad data: Numbers to sum:

line 2 has bad data: 

Average is 14.70


In [38]:
!ls /Users/stuartmoore/*.txt

/Users/stuartmoore/nums.txt         /Users/stuartmoore/requirements.txt


In [36]:
!pwd

/Users/stuartmoore/IDrive-Sync/pythonscratch


In [40]:
cat /Users/stuartmoore/nums.txt

Numbers to sum:

23.4
16.5
4.2

In [41]:
a=23.4+16.5+4.2
a/3

14.700000000000001

In [2]:

while True:
    response = input('Enter guess: ')
    try:
        user_input = int(response)
        break
    except ValueError as e:
        print('Please enter an integer')

Enter guess: 
Please enter an integer
Enter guess: hhh
Please enter an integer
Enter guess: 45.6
Please enter an integer
Enter guess: 456789
456789


In [ ]:
#! /usr/bin/env python3                                                                                                
# rock paper scissors Spock lizard game, rules follow:
'''                                                                                                                    
scissors cuts paper                                                                                                    
paper covers rock                                                                                                      
rock crushes lizard                                                                                                    
lizard poisons Spock                                                                                                   
Spock smashes scissors                                                                                                 
scissors decapitates lizard                                                                                            
lizard eats paper                                                                                                      
paper disproves Spock                                                                                                  
Spock vaporizes rock                                                                                                   
rock crushes scissors                                                                                                  
'''

from random import choice
import readline

RULES = list(map(str.split, __doc__.lower().strip().split('\n')))

OPTIONS = ({winner for winner, verb, loser in RULES}
           | {loser for winner, verb, loser in RULES})

PROMPT = f"Make your choice from: {', '.join(sorted(OPTIONS))} \n " \
            f"(or press return alone to exit)\n" \
            f" choice: "
                                     
def check(playera, playerb, rules=RULES):
    for rule in rules:
        winner, verb, loser = rule
        if (playera, playerb) == (winner, loser):
            return playera, ' '.join(rule)
        if (playerb, playera) == (winner, loser):
            return playerb, ' '.join(rule)

print('\n\nWelcome to the game of rock paper scissors Spock lizard\n\nThe rules are:\n')
print(__doc__)
print()
while True:
    while True:
        player = input(PROMPT).lower()
        if not player or player in OPTIONS:
            break
    if not player:
        break
    computer = choice(list(OPTIONS))    
    try:
        winner, rule = check(player, computer)
        result = 'You WIN!' if player == winner else 'You Lose!'
    except TypeError as e:
        result, rule = "TIED", 'matched'
    print(f'{player} v. {computer} -> {result} \t{rule}')
    print()

In [ ]:
UNITS = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]
TENS = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]
SCALES = ["hundred", "thousand", "million", "billion", "trillion"]
number_words = {"and": (1, 0)}
for position, word in enumerate(UNITS):
    number_words[word] = (1, position)
for position, word in enumerate(TENS):
    number_words[word] = (1, position * 10)
for position, word in enumerate(SCALES):
    number_words[word] = (10 ** (position * 3 or 2), 0)

POSITION_WORDS = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
SUFFIXES = [('ieth', 'y'), ('th', '')]

In [ ]:
def words_to_num(words):
    words = words.replace('-', ' ')
    words = words.replace(',', ' ')
    current = result = 0
    for word in words.strip().split():
        if word in POSITION_WORDS:
            scale, increment = 1, POSITION_WORDS[word]
        else:
            for ending, replacement in SUFFIXES:
                if word.endswith(ending):
                    word = f"{word[:-len(ending)]}{replacement}"
            if word not in number_words:
                raise ValueError(f"Illegal word: {word}")
            scale, increment = number_words[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0
    return result + current

In [42]:
def num_to_words(num):
    

In [43]:
print(words_to_num("four thousand, one hundred, fifty-six"))

4156


In [52]:

# Input and Data

quantity = float(input("Enter the quantity of items sold: "))
value = float(input("Enter the vaule of the item sold: "))
disc = float(input("Enter the discount percentage: "))
tax = float(input("Enter the tax: "))

# Calculations

amt = float(quantity * value)
disc_amt = float(amt * disc / 100)
sub_tot = float(amt - disc_amt)
tax_amt = float(sub_tot * tax / 100)
tot_amt = float(sub_tot + tax_amt)

# Output
bill = []
bill.append(("Quantity sold: ", "#", quantity, ''))
bill.append(("Price per item: ", "$", value, ''))
bill.append(("Amount: ", "$", amt, ''))
bill.append(("Discount: ", '$', disc_amt, ''))
bill.append(("Discount Total: ", '$', sub_tot, ''))
bill.append(("Tax: ", '$', tax_amt, ''))
bill.append(("Total amount to be paid: ", '$', tot_amt, ''))

desc_width = max([len(desc) for desc, _, _, _ in bill])

print("*********BILL*********")
for desc, pre, amount, pst in bill:
    print(f'{desc:<{desc_width}}{pre}{amount:7.2f}{pst}')
          

Enter the quantity of items sold: 10
Enter the vaule of the item sold: 23.45
Enter the discount percentage: 5.6
Enter the tax: 5
*********BILL*********
Quantity sold:           #  10.00
Price per item:          $  23.45
Amount:                  $ 234.50
Discount:                $  13.13
Discount Total:          $ 221.37
Tax:                     $  11.07
Total amount to be paid: $ 232.44


In [57]:
__author__ = 'anon'
for i in range(1, 12):
    print(f"No {i:2} squared is {i**2:3} and cubed is {i**3:4}")

No  1 squared is   1 and cubed is    1
No  2 squared is   4 and cubed is    8
No  3 squared is   9 and cubed is   27
No  4 squared is  16 and cubed is   64
No  5 squared is  25 and cubed is  125
No  6 squared is  36 and cubed is  216
No  7 squared is  49 and cubed is  343
No  8 squared is  64 and cubed is  512
No  9 squared is  81 and cubed is  729
No 10 squared is 100 and cubed is 1000
No 11 squared is 121 and cubed is 1331


In [62]:
def check_value(msg, start=0, fini=100):
    while True:
        response = input(msg)
        if response:
            try:
                value = int(response)
                if start <= value <= fini:
                    break
            except ValueError as e:
                pass
        print(f'Please enter a number between {start} and {fini}')
    return value

In [64]:
print(check_value('Num:', -100, -50))

Num:34
Please enter a number between -100 and -50
Num:-20
Please enter a number between -100 and -50
Num:-55
-55


In [70]:
numbers = []
nums_to_reverse = []
while True:
    response = input('Enter next number, or just return to finish: ')
    if not response:
        break
    try:
        numbers.insert(0, int(response))
        nums_to_reverse.append(int(response))
    except ValueError as e:
        print('Invalid response. Try again.')
print(*numbers, sep=", ")
print(*reversed(nums_to_reverse), sep=", ")

Enter next number, or just return to finish: 10
Enter next number, or just return to finish: 20
Enter next number, or just return to finish: 30
Enter next number, or just return to finish: 
30, 20, 10
30, 20, 10


In [68]:
numbers = []
while True:
    response = input('Enter next number, or just return to finish: ')
    if not response:
        break
    try:
        numbers.append(int(response))
    except ValueError as e:
        print('Invalid response. Try again.')
print(*reversed(numbers), sep=", ")

Enter next number, or just return to finish: 10
Enter next number, or just return to finish: 20
Enter next number, or just return to finish: 30
Enter next number, or just return to finish: 
30, 20, 10


In [71]:
    word  = "good_game"
    for character in word:
        print(character)
        word = word.replace(character,"G")
    print(word)

g
o
o
d
_
g
a
m
e
GGGGGGGGG


In [72]:
''' simple calculations using basic operators and input validation v2'''

    LABELS = {int: "integer", float: "floating point number", str: "string"}

    def get_input(msg = '', type_req = int):
        ''' prompt user for input with msg and return appropriately cast value '''   
        if not LABELS.get(type_req):
                raise TypeError("Unknown input type requested by calling code.")
        while True:
            response = input(msg)
            if response:
                try:
                    value = type_req(response)
                except ValueError as e:
                    pass
                else:
                    return value
            print(f'{LABELS[type_req]} input required. Please try again.')


num_x = get_input("enter an integer for x: ")
num_y = get_input("enter a float (decimal) for y: ", float)
print(f"x + y = {num_x + num_y}")
print(f"x - y = {num_x - num_y}")
print(f"x * y = {num_x * num_y}")
try:
    print(f"x / y = {num_x / num_y:.4f}")
except ZeroDivisionError as e:
    print('x / y not possible with 0 divisor')

enter an integer for x: 
integer input required. Please try again.
enter an integer for x: 45.6
integer input required. Please try again.
enter an integer for x: hello
integer input required. Please try again.
enter an integer for x: 10
enter a float (decimal) for y: 
floating point number input required. Please try again.
enter a float (decimal) for y: hello
floating point number input required. Please try again.
enter a float (decimal) for y: 45.6
x + y = 55.6
x - y = -35.6
x * y = 456.0
x / y = 0.2193


In [93]:
    username = input("Please enter your username: ")
    password = input("Please enter your password: ")

    result = 0
    if username and password:
        with open("passwords.txt") as f:
            for line in f:
                user, _ , pwd = line.strip().partition(";")
                result = ((user == username) + (pwd == password)) or result
                if result == 2:
                    break
    if result == 2:
        print("Welcome!")
    elif result:
        print("Wrong username or password")
    else:
        print("Not Found!")

Please enter your username: username3
Please enter your password: password3
Welcome!


In [78]:
!pwd

/Users/stuartmoore/IDrive-Sync/pythonscratch


In [108]:
    test_list = list(range(1900, 1911))
    for year in reversed(test_list):
        if year % 4:
            test_list.remove(year)
    print('using reversed:', test_list)

    test_list = list(range(1900, 1911))
    for year in test_list[:]:
        if year % 4:
            test_list.remove(year)
    print('using shallow copy:', test_list)

    test_list = list(range(1900, 1911))
    for idx in range(len(test_list) -1 , -1, -1):
        if test_list[idx] % 4:
            test_list.pop(idx)
    print('using shallow copy:', test_list)

using original [1900, 1902, 1904, 1906, 1908, 1910]
using reversed: [1900, 1904, 1908]
using shallow copy: [1900, 1904, 1908]
using shallow copy: [1900, 1904, 1908]


In [ ]:
def writelines(lines):
    print('writing')
    with open('random_new.txt', 'a') as f:
        for line in lines:
            f.write(line.strip() + " is an excellent webcomic\n")
        del lines[:]

EMPTY = 3  # number of lines to process per batch
lines = []
with open('random.txt') as f:
    for line in f:
        lines.append(line)
        if len(lines) >= EMPTY:
            writelines(lines)

In [87]:
''' temperature conversion '''

def tempconv_F_to_C(temp):
    return (temp - 32) / 1.8

def tempconv_C_to_F(temp):
    return (temp * 1.8) + 32

def tempconv_Ré_to_C(temp):
    return temp * 5/4

def tempconv_C_to_Ré(temp):
    return temp * 4/5

def tempconv_Ré_to_F(temp):
    return tempconv_C_to_F(tempconv_Ré_to_C(temp))

def tempconv_F_to_Ré(temp):
    return tempconv_C_to_Ré(tempconv_F_to_C(temp))

# build dictionary of temperature conversion functions, 
# (dropping from the keys the prefix of tempconv_ and removing the _ before and after to)
prefix = 'tempconv_'
prefix_len = len(prefix)
tempconv_funcs = {name[prefix_len:].replace('_to_', ' to '): func for name, func in globals().items() 
                 if callable(func) and name.startswith(prefix)
                }
# define set of temperature units available
UNITS = set([unit for temp in [k.partition(' to ') 
                for k in tempconv_funcs.keys()] 
                for unit in temp if unit != ' to '])
ORIG = 'C'  # default original temp
NEW = 'F'  # default to convert to

def temp_convert(temp, orig, new):
    if not(isinstance(temp, int) or isinstance(temp, float)):
        raise ValueError('Valid temperature not provided')
    if not orig or not new:
        raise ValueError('Original and/or New units not specified')
    func = tempconv_funcs.get(f'{orig} to {new}')
    if not func:
        raise TypeError(f'No coversion available for {orig} to {new}')
    return func(temp)

print('Welcome to this temperature conversion programme\n')
print(f'The following units are accepted: {", ".join(UNITS)}')
print('and the following conversions:-', end='')
print('', *tempconv_funcs.keys(), sep='\n * ')

while True:  # do another conversion loop
    while True:  # get valid unit selections from user 
        orig = input(f'What units to convert from? [{ORIG}]  ')
        if not orig:
            orig = ORIG
        new = input(f'What units to convert to? [{NEW if orig != NEW else ORIG}] ')
        if not new:
            new = NEW if orig != NEW else ORIG
        if orig != new and orig in UNITS and new in UNITS:
            break
        print(f'You need to select from the following units: {", ".join(UNITS)}')

    while True:  # get a numeric input for temperature
        try:
            temp = float(input(f'Temperature ({orig}): '))
            break
        except:
            print('Numeric value required')

    try:
        print(f'{temp:.2f} degrees {orig} is {temp_convert(temp, orig, new):.2f} degrees {new}')
    except ValueError as e:
        print(e)
    except TypeError as e:
        print(e)

    if input('Another? [Yes/no] ').lower() not in ('y', 'yes', '1', 'yup', 'ok', ''):
        break

Welcome to this temperature conversion programme

The following units are accepted: F, C, Ré
and the following conversions:-
 * F to C
 * C to F
 * Ré to C
 * C to Ré
 * Ré to F
 * F to Ré
What units to convert from? [C]  
What units to convert to? [F] 
Temperature (C): 100
100.00 degrees C is 212.00 degrees F
Another? [Yes/no] n


In [84]:
tempconv_funcs.keys()

dict_keys(['F to C', 'C to F', 'Ré to C', 'C to Ré', 'Ré to F', 'F to Ré'])

In [86]:
prefix_len

9

In [91]:
    def double(x):
        return x * 2

    print(5, double(3))

    def discount(unit_price, quantity):
        'return discount $ amount, if any'
        total = unit_price * quantity
        if total > 20 and quantity > 4:
            disc = total * 10 / 100
        else:
            disc = 0
        return disc

    ticket = float(input('How much each? '))
    qty = int(input('How many? '))

    to_pay = ticket * qty - discount(price, qty)

    print('Pay:', to_pay)

5 6
How much each? 100
How many? 20
Pay: 1980.0


In [106]:
    def find_num(data, val):
        low = 0
        high = len(data) - 1  # we count from zero
        while high - low > 0 and data[low] != val:
            mid = int((high - low) / 2 ) + low
            if data[mid] == val: 
                low = mid
            elif val < data[mid]:
                high = mid - 1
            else:
                low = mid + 1
        return low if data[low] == val else -1

In [108]:
nums = [2, 4, 6, 7, 8, 10, 12]
for x in range(max(nums) + 3):
    print(f'found {x} in position {find_num(nums, x)}')

found 0 in position -1
found 1 in position -1
found 2 in position 0
found 3 in position -1
found 4 in position 1
found 5 in position -1
found 6 in position 2
found 7 in position 3
found 8 in position 4
found 9 in position -1
found 10 in position 5
found 11 in position -1
found 12 in position 6
found 13 in position -1
found 14 in position -1


In [114]:
print(list(range(10)))

nums = [x for x in range(10)]
print(nums)

nums = []
for idx in range(10):
    nums.append(idx)
print(nums)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


(Financial application: compound value) Suppose you save $100 each month into a savings account with an annual interest rate of 5%. Therefore, the monthly interest rate is After the first month, the value in the account becomes 100 * (1 + 0.00417) = 100.417 After the second month, the value in the account becomes (100 + 100.417) * (1 + 0.00417) = 201.252 After the third month, the value in the account becomes (100 + 201.252) * (1 + 0.00417) = 302.507 and so on. Write a program that prompts the user to enter a monthly saving amount and displays the account value after the sixth month. Here is a sample run of the program:

Enter the monthly saving amount: (WHATEVER NUMBER)

After the sixth month, the account value is (WHATEVER NUMBER)

In [117]:
saving = float(input('Amount to save each month? '))
balance = saving
rate = 5 # percent
monthly_rate = rate / 12.0
for month in range(1, 7):
    interest = balance * monthly_rate / 100.0
    balance += interest
    print(f'End of month: {month}, interest earned: {interest:.2f}, balance: {balance:.2f}')
    balance += saving

Amount to save each month? 100
End of month: 1, interest earned: 0.42, balance: 100.42
End of month: 2, interest earned: 0.84, balance: 201.25
End of month: 3, interest earned: 1.26, balance: 302.51
End of month: 4, interest earned: 1.68, balance: 404.18
End of month: 5, interest earned: 2.10, balance: 506.28
End of month: 6, interest earned: 2.53, balance: 608.81


In [121]:
def test1():
    return 1
 
def test2():
    return 2
 
def test3():
    return 3
 
def testall(x, y, z):
    print(x)
    print(y)
    print(z)
 
if __name__ == '__main__':
    x = test1()
    y = test2()
    z = test3()
    testall(x, y, z)

SyntaxError: invalid syntax (<ipython-input-121-e6b903452971>, line 1)

In [123]:
odds = ''.join([str(x) for x in range(1, 10, 2)])
evens = ''.join([str(x) for x in range(0, 10, 2)])
data = ['124678', '1134546', '9091102']
output = [''.join([ch for ch in entry if ch in evens] + [ch for ch in entry if ch in odds]) for entry in data]
print(output)

['246817', '4461135', '0029911']


In [124]:
?str.replace

In [129]:
class _Getch:
    """Gets a single character from standard input.  Does not echo to the
screen."""
    def __init__(self):
        try:
            self.impl = _GetchWindows()
        except ImportError:
            self.impl = _GetchUnix()
 
    def __call__(self): return self.impl()
 
 
class _GetchUnix:
    def __init__(self):
        import tty, sys
 
    def __call__(self):
        import sys, tty, termios
        fd = sys.stdin.fileno()
        old_settings = termios.tcgetattr(fd)
        try:
            tty.setraw(sys.stdin.fileno())
            ch = sys.stdin.read(1)
        finally:
            termios.tcsetattr(fd, termios.TCSADRAIN, old_settings)
        return ch
 
 
class _GetchWindows:
    def __init__(self):
        import msvcrt
 
    def __call__(self):
        import msvcrt
        return msvcrt.getch()
 
 
getch = _Getch()#origin
char = getch.__call__()

error: (25, 'Inappropriate ioctl for device')

In [131]:
    while True:
        choice = input('Enter choice: ')
        if choice in "1234":
            break

    if choice == "1":
        pass
    elif choice == "4":
        nums = []
        while True:
            variabel = input("Write a number: ")
            if variabel == "done":
                break
            try:
               nums.append(float(variabel))
            except:
                print('Invalid entry')
        sumofallnumbers = sum(nums)
        averagenumber = sumofallnumbers/len(nums)
        print(sumofallnumbers)
        print(averagenumber)

Enter choice: 4
Write a number: 1
Write a number: 2
Write a number: 3
Write a number: 4
Write a number: 5
Write a number: done
15.0
3.0


In [137]:
    import string
    ALLOWED = set(string.ascii_lowercase + '.' + "'" + ' ' )

    def name_valid(test_str):
        return set(test_str).issubset(ALLOWED)

    while True:
        name = input('Enter name: ')
        if name_valid(name.lower()):
            break
        print('Does not look like a name to me. Try again.')

    print('Hello', name)

Enter name: Fred
Hello Fred


In [140]:
while True:    
    response = input('Some prompt: ')
    try:
        num = float(response)
        break  # execution breaks out of loop
    except ValueError as e:
        print('Please enter a number')

print(f'You entered {num}')

Some prompt: 
Please enter a number
Some prompt: hjhjh
Please enter a number
Some prompt: 56.7
You entered 56.7


In [149]:
largest = - 999999999999
smallest = 999999999999
while True:
    num = input("Enter a number: ")
    if num == "done":
        break
    try:
        entryStuff = float(num)
    except:
        print("Invalid input")
    else:
        if entryStuff > largest:
            largest = entryStuff
        if entryStuff < smallest:
            smallest = entryStuff
 
print("Maximum", largest)
print("Minimum", smallest)

Enter a number: 10
Enter a number: 20
Enter a number: 5
Enter a number: 30
Enter a number: 25
Enter a number: done
Maximum 30.0
Minimum 5.0


In [152]:
a=10
list(range(1,a+1))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [154]:
    grades = []
    while True:
        grade = input('Enter grade, or just return to finish: ')
        if not grade:
            break
        grades.append(int(grade))
    print(grades)  # just outputting the list here, you need code to calc averages

Enter grade, or just return to finish: 10
Enter grade, or just return to finish: 20
Enter grade, or just return to finish: 34
Enter grade, or just return to finish: 
[10, 20, 34]


In [155]:
    import readchar  # https://pypi.org/project/readchar/



    MENU_ENTRIES = '12xX'

    while True:

        print('Press 1 for Squares, Press 2 for triangles, Press x to exit')

        while True:
            key = readchar.readkey()
            if key in MENU_ENTRIES:
                break

        if key.lower() == 'x':
            break

        length = float(input('Enter length: '))
        height = float(input('Enter heigth '))

        if key == "1":
            print(f'Square of length {length} x height {height} has area of {length * height}')

        elif key == "2":
            print(f'Triangle of length {length} x height {height} has area of {length * height / 2}')

Enter length: 10
Enter heigth 5
Press 1 for Squares, Press 2 for triangles, Press x to exit


error: (25, 'Inappropriate ioctl for device')

In [164]:
def is_prime(num):
    if num == 1: return False
    if num in (2, 3): return True
    if not num % 2: return False
    for value in range(3, int(num**0.5+1), 2):
        if not num % value: return False
    return True

![formulae](https://cs.uwaterloo.ca/~alopez-o/math-faq/img297.gif)

k is day (1 to 31)

m is month (1 = March, ..., 10 = December, 11 = Jan, 12 = Feb) Treat Jan & Feb as months of the preceding year

C is century (1987 has C = 19)

Y is year (1987 has Y = 87 except Y = 86 for Jan & Feb)

W is week day (0 = Sunday, ..., 6 = Saturday)

From: https://cs.uwaterloo.ca/~alopez-o/math-faq/node73.html

In [213]:
''' determine day of week for any Gregorian date '''

# based on formulae at https://cs.uwaterloo.ca/~alopez-o/math-faq/node73.html

DOW = dict(enumerate(('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday')))
MONTHS = dict(enumerate(('January', 'February', 'March', 'April',
                       'May', 'June', 'July', 'August',
                       'September', 'October', 'November', 'December'), start=1))



def is_leap_year(year):
    'return True/False if year is a leap year'
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def get_date(msg):
    'get date from user and return tuple of dd, mm, yyyy'
    while True:
        response = input(msg).strip().split('/')
        if response and len(response) == 3:
            try:
                dd, mm, yyyy = [int(x) for x in response]
                if not 1 <= dd <= 31:
                    raise IndexError('Day must be between 1 and 31')
                if not 1 <= mm <= 12:
                    raise IndexError('Month must be between 1 and 12')
                if not 1582 <= yyyy <= 9999:
                    raise IndexError('Year must be between 1582 and 9999')
                if mm == 2 and dd > 28:
                    if not is_leap_year(yyyy) or dd > 29:
                        raise IndexError(f'Too many days for {MONTHS[2]} in {yyyy}')
                if dd == 31 and mm not in (1, 3, 5, 7, 8, 10, 12):
                    raise IndexError(f'Too many days for {MONTHS[mm]}')
                return dd, mm, yyyy
            except ValueError as e:
                print(f'{response} can not be understood as a date such as 18/05/1958')
            except IndexError as e:
                print(e)
        else:
            print('A response in the required format is expected. Please try again.')

def day_of_week(dd, mm, yyyy):
    'return day of week for date dd/mm/yyyy'
    mm = mm - 2  # march = 1, jan = 11
    if mm < 1:
        mm + 12
    if mm >= 11:  # year - 1 if month is jan or feb
        yyyy -= 1

    C = int(yyyy / 100)
    Y = yyyy % (C * 100)
    W = (dd + int(2.6 * mm - 0.2) - 2 * C + Y + int(Y/4) + int(C/4)) % 7

    return DOW[W]

dd, mm, yyyy = get_date('Please enter date of birth in form: dd/mm/yyyy: ')
print(f'{dd}/{mm}/{yyyy} is a {day_of_week(dd, mm, yyyy)}')

Please enter date of birth in form: dd/mm/yyyy: 14/9/2018
14/9/2018 is a Friday


In [200]:
DOW

{0: 'Sunday',
 1: 'Monday',
 2: 'Tuesday',
 3: 'Wednesday',
 4: 'Thursday',
 5: 'Friday',
 6: 'Saturday'}

In [205]:
MONTHS

{1: 'January',
 2: 'February',
 3: 'March',
 4: 'April',
 5: 'May',
 6: 'June',
 7: 'July',
 8: 'August',
 9: 'September',
 10: 'October',
 11: 'November',
 12: 'December'}